In [ ]:
import pandas as pd
import geopandas as gpd
import os
import gc
import numpy as np
from scipy import stats
from shapely.geometry import Point
import matplotlib.pyplot as plt
from matplotlib.colors import to_hex
import matplotlib.animation as animation
import matplotlib.image as mpimg
from plotnine import *
plt.rcParams['font.family'] = "Noto Sans CJK JP"

def make_file_list(folder_path, file_ext):
    file_list = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(file_ext)]
    return file_list

def bind_df(file_list,encoding = 'utf-8',kind = 'df', epsg = 4326):
    if kind == 'df':
        df = pd.DataFrame()
        for file_name in file_list:
            data = pd.read_csv(file_name,encoding = encoding)
            df = pd.concat([df, data])
            df['ID'] = range(1,len(df.index)+1)
        return df
    elif kind == 'gdf':
        gdf = gpd.GeoDataFrame()
        for file_name in file_list:
            data = gpd.read_file(file_name,encoding = encoding)
            data = data.to_crs(epsg = epsg)
            gdf = pd.concat([gdf, data])
            gdf['ID'] = range(1,len(df.index)+1)
        return gdf

def make_list(data,column_name):
    dataframe =  data[[column_name]].drop_duplicates()
    List = dataframe[column_name].tolist()
    return List

def get_quantiles(data, num_quantiles=4):
    quantiles = [i / (num_quantiles - 1) for i in range(num_quantiles)]
    return data.quantile(quantiles).tolist()

def dms_to_dd(dms,digit,column_type):
    try:
        dms_str = str(dms).strip().zfill(digit)
        if column_type == 'lat':
            degrees = int(dms_str[:2])
            minutes = int(dms_str[2:4])
            seconds = int(dms_str[4:])
            dd = degrees + (minutes / 60.0) + (seconds / (10 ** (digit - 6)) / 3600.0)
            return dd
        elif column_type == 'lon':
            degrees = int(dms_str[:3])
            minutes = int(dms_str[3:5])
            seconds = int(dms_str[5:])
            dd = degrees + (minutes / 60.0) + (seconds / (10 ** (digit - 7)) / 3600.0)
            return dd
        else:
            raise ValueError("column_type must be either 'lat' or 'lon'")
    except ValueError:
        return None

def add_xy(data, lon_col, lat_col, lon_digit, lat_digit, Japan_land = True):
    data = data[data[lon_col].apply(lambda x: str(x).strip().replace(' ', '').isdigit())]
    data = data[data[lat_col].apply(lambda x: str(x).strip().replace(' ', '').isdigit())]
    data['lon'] = data[lon_col].apply(lambda x: dms_to_dd(x, lon_digit, 'lon'))
    data['lat'] = data[lat_col].apply(lambda x: dms_to_dd(x, lat_digit, 'lat'))
    data = data.dropna(subset=['lon', 'lat'])
    if Japan_land is True:
        data = data[(data['lon'] > 120) & (data['lon'] < 155)]
        data = data[(data['lat']> 20) & (data['lat'] < 46)]
    data['geometry'] = [Point(xy) for xy in zip(data['lon'], data['lat'])]
    return data

def count_point(polygon, point, group_by = None, predicate = 'contains'):
    point['ID'] = range(1,len(point.index)+1)
    polygon_with_point = gpd.sjoin(polygon, point, how = 'inner', predicate = predicate)
    point_by_polygon = polygon_with_point.groupby(group_by)['ID'].apply(lambda x: ', '.join(x.dropna().astype(str))).reset_index()
    polygon['point_ID'] = polygon.merge(point_by_polygon, left_on=group_by, right_on = group_by, how='left')['ID']
    polygon['point_count'] = polygon['point_ID'].apply(lambda x: len(x.split(',')) if isinstance(x, str) else 0)
    return polygon

def set_theme():
    return theme(text=element_text(family='Noto Sans CJK JP'),
                 panel_background=element_rect(fill='white'), 
                 plot_background=element_rect(fill='white'),
                 panel_grid_major=element_blank(), 
                 panel_grid_minor=element_blank(),
                 axis_title=element_blank(),
                 axis_text=element_blank(),
                 axis_ticks=element_blank())

def make_choro_core(polygon,
                    polygon_column = None,
                    border_color = None,
                    cmap='jet',
                    binning ='quantile',
                    bins=5):
    if polygon_column is not None:
        if binning not in ['quantile', 'equal']:
            raise ValueError("binning must be 'quantile' or 'equal'")
        if binning == 'quantile':
            polygon['legend'] = pd.qcut(polygon[polygon_column], bins)
        elif binning == 'equal':
            polygon['legend'] = pd.cut(polygon[polygon_column], bins, right = False)
        if isinstance(bins, list):
            bins = len(bins)
        cmap = plt.get_cmap(cmap, bins)
        cmap_colors = [to_hex(cmap(i/bins)) for i in range(bins)]
        return geom_map(data = polygon, mapping = aes(fill = 'legend'),color = border_color), cmap_colors
    else :
        return geom_map(data = polygon,fill = None, color = border_color), None

def make_choro(polygon,
               polygon_column = None,
               legend_title = None,
               border_color = None,
               cmap='jet',
               binning ='quantile',
               bins=5,
               title = None,
               save = False,
               file_name = None,
               folder_path = None):
    try:
        polygon_layer, cmap_colors = make_choro_core(polygon,
                                                     polygon_column,
                                                     border_color,
                                                     cmap,
                                                     binning,
                                                     bins)
        choro = (ggplot() +
                 set_theme() +
                 polygon_layer +
                 scale_fill_manual(values=cmap_colors) +
                 labs(fill=legend_title) +
                 labs(title=title) +
                 coord_equal())
        if save is True:
            save_path = folder_path + file_name + ".png"
            choro.save(save_path)
        else:
            print(choro)
        del choro
        gc.collect()
    except ValueError as e:
        error_path = folder_path + file_name + ".txt"
        with open(error_path, 'w') as f:
            f.write('creating choro is failed:\n')
            f.write(f'ValueError:{str(e)}')

def make_dot_core(point,
                  point_shape = 'o',
                  fill_point_color = 'red',
                  point_color = None,
                  point_size = 2,
                  label_column = None,
                  label_size = 10):
    dot = geom_map(data = point,
                   shape = point_shape,
                   fill = fill_point_color
                   ,color = point_color,
                   size = point_size)
    if label_column is not None:
        point['lon'] = point.geometry.x
        point['lat'] = point.geometry.y
        dot_label = geom_text(point, aes(x = 'lon',y = 'lat',label= label_column), size= label_size, ha = 'left')
        return dot, dot_label
    else :
        return dot, None

def make_dot_dist(point,
                  polygon = None,
                  polygon_column = None,
                  legend_title = None,
                  border_color = 'black',
                  cmap='jet',
                  binning ='quantile',
                  bins=5,
                  point_shape = 'o',
                  fill_point_color = 'red',
                  point_color = None,
                  point_size = 2,
                  label_column = None,
                  label_size = 10,
                  title = None,
                  save = False,
                  file_name = None,
                  folder_path = None):
    if len(point) == 0:
        if save is False:
            raise ValueError('point内にデータがありません')
        else:
            error_path = folder_path + file_name + ".txt"
            with open(error_path, 'w') as f:
                f.write('creating dot is failed:\n')
                f.write(f'ValueError:point内にデータがありません')
    try:
        point_layer, point_label = make_dot_core(point,
                                                 point_shape,
                                                 fill_point_color,
                                                 point_color,
                                                 point_size,
                                                 label_column,
                                                 label_size)
        dot_dist = ggplot() + set_theme()
        if polygon is not None:
            polygon_layer, cmap_colors = make_choro_core(polygon,
                                                         polygon_column,
                                                         border_color,
                                                         cmap,
                                                         binning,
                                                         bins)
            dot_dist += polygon_layer
            if polygon_column is not None:
                dot_dist += scale_fill_manual(values=cmap_colors)
        dot_dist += point_layer
        if point_label is not None:
            dot_dist += point_label
        dot_dist += labs(title=title, fill=legend_title) + coord_equal()
        if save is True:
            save_path = folder_path + file_name + ".png"
            dot_dist.save(save_path)
        else:
            print(dot_dist)
        del dot_dist
        gc.collect()
    except ValueError as v:
        error_path = folder_path + file_name + ".txt"
        with open(error_path, 'w') as f:
            f.write('creating dot is failed:\n')
            f.write(f'ValueError:{str(v)}')

def make_prop(polygon,
              size_column,
              polygon_column = None,
              polygon_title = None,
              size_title = None,
              border_color = None,
              cmap='jet',
              binning ='quantile',
              bins=5,
              point_shape = 'o',
              point_color = 'red',
              size_times = 1,
              point_quantiles = 4,
              title = None,
              save = False,
              file_name = None,
              folder_path = None):
    try:
        point = polygon.copy()
        point['geometry'] = polygon.geometry.representative_point()
        point = gpd.GeoDataFrame(point, geometry = 'geometry')
        point = point[point[size_column] > 0]
        point['x'] = point.geometry.x
        point['y'] = point.geometry.y
        point = point.drop(columns='geometry')
        polygon_layer, cmap_colors = make_choro_core(polygon,
                                                    polygon_column,
                                                    border_color,
                                                    cmap,
                                                    binning,
                                                    bins)
        prop = (ggplot() +
               set_theme() +
               polygon_layer)
        if polygon_column is not None:
                prop += scale_fill_manual(values=cmap_colors) 
        if point_color is None:
            raise ValueError('Error:point_colorにNoneは指定できません')
        point[size_column] = point[size_column] * size_times
        prop += geom_point(point,aes("x", "y", size=size_column),
                           shape = point_shape,
                           color = point_color)
        quantiles = get_quantiles(point[size_column], num_quantiles= point_quantiles)
        prop += scale_size_continuous(breaks = quantiles)
        prop += labs(fill = polygon_title, size = size_title,title = title) + coord_equal()
        if save is True:
            save_path = folder_path + file_name + ".png"
            prop.save(save_path)
        else:
            print(prop)
        del prop
        gc.collect()
    except ValueError as e:
        error_path = folder_path + file_name + ".txt"
        with open(error_path, 'w') as f:
            f.write('creating dot is failed:\n')
            f.write(f'ValueError:{str(e)}')

def make_kde(point,
             polygon = None,
             boundary = 'point',
             bandwidth = 1000,
             epsg = 3857,
             buffer = 0,
             resolution = 500,
             cmap = 'jet',
             num_colors = 5,
             kde_alpha = 1.0,
             point_plot = False,
             point_shape = 'o',
             fill_point_color = 'red',
             point_color = None,
             point_size = 2,
             title = None,
             save = False,
             file_name = None,
             folder_path = None):

    if len(point) == 0:
        if save is False:
            raise ValueError('point内にデータがありません')
        else:
            error_path = folder_path + file_name + ".txt"
            with open(error_path, 'w') as f:
                f.write('creating kde is failed:\n')
                f.write(f'ValueError:point内にデータがありません')
    if point.crs.is_geographic:
        point = point.to_crs(epsg = epsg)
        if point.crs.is_geographic:
            raise ValueError('Error:epsgは投影座標系を指定してください')
    if boundary == 'point':
        # グリッドを作成
        xmin, ymin, xmax, ymax = point.total_bounds
        xmin = xmin - abs(xmin * buffer)
        ymin = ymin - abs(ymin * buffer)
        xmax = xmax + abs(xmax * buffer)
        ymax = ymax + abs(ymax * buffer)
        xgrid = np.linspace(xmin, xmax, resolution)  # 解像度を500x500に設定
        ygrid = np.linspace(ymin, ymax, resolution)
        xv, yv = np.meshgrid(xgrid, ygrid)

    elif boundary == 'polygon':
        # グリッドを作成
        xmin, ymin, xmax, ymax = polygon.total_bounds
        xmin = xmin - abs(xmin * buffer)
        ymin = ymin - abs(ymin * buffer)
        xmax = xmax + abs(xmax * buffer)
        ymax = ymax + abs(ymax * buffer)
        xgrid = np.linspace(xmin, xmax, resolution)  # 解像度を500x500に設定
        ygrid = np.linspace(ymin, ymax, resolution)
        xv, yv = np.meshgrid(xgrid, ygrid)
    # データを準備
    x = point.geometry.x
    y = point.geometry.y
    values = np.vstack([x, y])
    
    # カスタム帯域幅の設定 よくわからないが、epsg:3857だと100をかけると同一の結果が得られる
    if epsg == 3857:
        band = bandwidth * 100
    else:
        band = bandwidth
    try:
        kernel = stats.gaussian_kde(values, bw_method=band / np.std(values, ddof=1))    
        # KDEの計算
        positions = np.vstack([xv.ravel(), yv.ravel()])
        kde_values = np.reshape(kernel(positions).T, xv.shape)
        # データフレームに変換
        df = pd.DataFrame({
            'x': xv.ravel(),
            'y': yv.ravel(),
            'z': kde_values.ravel()
        })
        # KDEの可視化
        KDE = (ggplot()+
               set_theme())
        if polygon is not None:
            polygon = polygon.to_crs(epsg = epsg)
            KDE += geom_map(data = polygon, fill = None, color = 'black')
        cmap = plt.get_cmap(cmap,num_colors)
        cmap_colors = [to_hex(cmap(i/num_colors)) for i in range(num_colors)]
        KDE += geom_tile(aes('x', 'y', fill='z'), data=df, alpha = kde_alpha)
        KDE += scale_fill_gradientn(colors= cmap_colors)
        if point_plot is True:
            point_layer, point_label = make_dot_core(point,
                                                     point_shape,
                                                     fill_point_color,
                                                     point_color,
                                                     point_size)
            KDE += point_layer
        KDE += (labs(title=title) +
                coord_equal())  
        if save is True:
            save_path = folder_path + file_name + ".png"
            KDE.save(save_path)
        else:
            print(KDE)
        del KDE
        gc.collect()
    except ValueError as e:
        if save is False:
            raise ValueError(f'{e}')
        else:
            error_path = folder_path + file_name + ".txt"
            with open(error_path, 'w') as f:
                f.write('creating kde is failed:\n')
                f.write(f'ValueError:{e}')

def make_ani(frame_files,file_path):
    fig = plt.figure()
    plt.axis('off')
    fig.subplots_adjust(left=0, right=1, top=1, bottom=0)
    imgs = [mpimg.imread(f) for f in frame_files]
    im = plt.imshow(imgs[0])
    def update(frame):
        im.set_array(frame)
        return [im]
    ani = animation.FuncAnimation(fig, update, frames=imgs, interval=1000)
    ani.save(file_path, writer="pillow", dpi=300)
    plt.clf()
    plt.close()
    del ani
    gc.collect()